# Importations

In [2]:
from gensim.models.word2vec import Word2Vec
import cPickle
import numpy as np
from math import sqrt
%matplotlib inline

# Training the model

In [3]:
model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Some tests

In [ ]:
model.most_similar(positive=['man', 'queen'], negative=['woman'], topn=5)

In [ ]:
# model.save_word2vec_format('GoogleNews-vectors-negative300.txt', binary=False)

In [4]:
model.syn0.shape

(3000000, 300)

In [6]:
word_vectors = model.syn0[:1000]

In [9]:
num_clusters = word_vectors.shape[0] / 5

In [5]:
from pyspark.mllib.clustering import KMeans

In [7]:
vectors = sc.parallelize(word_vectors)

In [10]:
idx = KMeans.train(vectors, num_clusters, maxIterations=10, runs=30, initializationMode="random")

In [11]:
word_centroid_map = dict(zip( model.index2word, idx.centers ))

In [12]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        for j in xrange(0,len(word_centroid_map.values())):
            if( sum(word_centroid_map.values()[i]) == sum(word_centroid_map.values()[j]) ):
                words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'all', u'help', u'just', u'being', u'percent', u'over', u'both', u'years', u'four', u'through', u'world', u'go', u'still', u'</s>', u'its', u'before', u'We', u'now', u'police', u'$', u'$', u'how', u'had', u'day', u'only', u'going', u'under', u'has', u'By', u'###', u'do', u'them', u'his', u'They', u'get', u'very', u'end', u'me', u'game', u'five', u'know', u'they', u'got', u'got', u'not', u'during', u'part', u'one', u'him', u'school', u'like', u'In', u'did', u"'ve", u'these', u'she', u'team', u'including', u'told', u'where', u'week', u'because', u'says', u'For', u'people', u'##', u'some', u'back', u'around', u'second', u'are', u'year', u'our', u'out', u'even', u'what', u'said', u'for', u'That', u'since', u'#-#', u'may', u'state', u'does', u'between', u'new', u'He', u'be', u'we', u'after', u'business', u'This', u'U.S.', u'million', u'based', u'by', u'on', u'on', u'about', u'last', u'her', u'##,###', u'many', u'could', u'against', u'or', u'first', u'into', u'There', u'three', 

TypeError: unhashable type: 'numpy.ndarray'

# Storing in a dict

In [ ]:
word2vec_dict={}
for i in model.vocab.keys():
    try:
        word2vec_dict[i]=model[i]
    except:
        pass

In [ ]:
f = file('dict.save', 'wb')
cPickle.dump(word2vec_dict, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
indextokey=dict()
i=0
for keys, values in word2vec_dict.iteritems():
    indextokey[i]=keys
    i+=1

In [ ]:
indextovalue=dict()
i=0
for keys, values in word2vec_dict.iteritems():
    indextovalue[i]=values
    i+=1

---------

# Loading the dict

In [ ]:
f = file('dict.save', 'rb')
word2vec_dict = cPickle.load(f)
f.close()

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering

In [ ]:
clusters = MiniBatchKMeans(n_clusters=100, max_iter=3,batch_size=200,
                        n_init=1,init_size=2000)
X = np.array([i.T for i in word2vec_dict.itervalues()])
y = [i for i in word2vec_dict.iterkeys()]
clusters.fit(X)
from collections import defaultdict
cluster_dict=defaultdict(list)
for word,label in zip(y,clusters.labels_):
    cluster_dict[label].append(word)

In [ ]:
type(model)

In [ ]:
model["house"]

# Using Spark

In [ ]:
from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans, KMeansModel

# Test SparkContext

In [ ]:
sc

#### cool, that works

In [ ]:
model_cluster = KMeans.train(b, 100, maxIterations=10, runs=30, initializationMode="random")

In [ ]:
sc

In [ ]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [ ]:
n_items = take(10000, indextovalue.itervalues())

In [ ]:
a=np.array(n_items).reshape(10000,300)

In [ ]:
b=sc.parallelize(a)

In [ ]:
a.flatMap(lambda x: x.items())

In [ ]:
a

In [ ]:
len(n_items[4])

In [ ]:
a=[item for sublist in n_items for item in sublist]

In [ ]:
b=sc.parallelize(a)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

ts = TSNE(2)
reduced_vecs = ts.fit_transform(model_cluster.clusterCenters)

In [ ]:
for i in range (len(reduced_vecs)):
    plt.plot(reduced_vecs[i,0], reduced_vecs[i,1], marker='.', markersize=8)

In [ ]:
def error(point):
    center = model_cluster.centers[model_cluster.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = b.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

In [ ]:
len(reduced_vecs)